In [8]:
#= Importy i załadowanie danych =#
using JLD2
using Printf
using Statistics
using LinearAlgebra

include("src/AD.jl")
include("src/NN.jl")

Main.NN

In [9]:
#= Przygotowanie danych =#
file = load("data/imdb_dataset_prepared.jld2")
X_train = Int.(file["X_train"])
y_train = vec(Float32.(file["y_train"]))
X_test  = Int.(file["X_test"])
y_test  = vec(Float32.(file["y_test"]))
embeddings = file["embeddings"]
vocab = file["vocab"]
embedding_dim = size(embeddings, 1)

50

In [10]:
dataset = NN.DataLoader((X_train, y_train), batchsize=64, shuffle=true)

Main.NN.DataLoader((Float32[6391.0 286.0 … 11382.0 8311.0; 143.0 286.0 … 4254.0 2891.0; … ; 12849.0 12849.0 … 12849.0 12849.0; 12849.0 12849.0 … 12849.0 12849.0], Float32[0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]), 64, true, [5796, 30389, 27281, 19486, 5858, 34381, 19827, 35975, 16630, 3721  …  10049, 37691, 14583, 11680, 906, 8656, 24433, 12814, 4813, 27508])

In [11]:
model = NN.Chain(
    x -> Int.(x),
    NN.Embedding(length(vocab), embedding_dim, embeddings),
    x -> permutedims(x, (2, 1, 3)),
    NN.Conv1D(3, embedding_dim, 8, AD.relu),
    NN.MaxPool1D(8),
    NN.flatten,
    NN.Dense(128, 1, AD.sigmoid)
)

Main.NN.Chain(Any[var"#5#7"(), Main.NN.Embedding(12849, 50, Float32[0.90951 -0.58014 … 0.014323 0.0; -0.20702 -1.1316 … -0.74624 0.0; … ; -0.60396 0.1722 … -0.079873 0.0; -0.31136 1.2622 … -0.89164 0.0], Int64[]), var"#6#8"(), Main.NN.Conv1D(3, 50, 8, Float32[-0.19059059 0.07762306 … 0.005278347 0.011101543; 0.02321488 0.13453364 … -0.033101495 -0.0043567033; -0.029989028 -0.108399674 … -0.009360174 -0.14080127;;; -0.0026544903 0.03309946 … 0.19797084 0.049405; -0.03671582 0.017825106 … 0.00391493 -0.1651523; 0.009887318 -0.15669845 … 0.0023309402 -0.0020684125;;; -0.05400781 -0.117357165 … -0.07769784 0.028266223; -0.06891862 0.013558016 … -0.06408801 0.3544704; -0.03977637 0.2068134 … -0.12930143 0.05978754;;; 0.13516054 0.13269058 … -0.16400383 0.033789992; 0.13248985 0.10158137 … 0.062744536 0.15652162; 0.061711874 -0.04589952 … 0.00525208 -0.09641091;;; 0.10978968 0.2912776 … -0.096927874 -0.19934873; 0.04203788 -0.14108819 … -0.1923494 -0.0721688; -0.09691382 0.18789165 … -0.0967

In [12]:
accuracy(m, x, y) = mean((vec(m(x)) .> 0.5) .== (y .> 0.5))

accuracy (generic function with 1 method)

In [14]:
#= Trening modelu =#
opt = NN.Adam(Float32(0.001))
epochs = 5
NN.train_model(model, dataset, X_test, y_test, opt, epochs)

Epoch: 1 (21.07s) 	Train: (l: 0.41, a: 0.82) 	Test: (l: 0.40, a: 0.82) 	Memory allocated: 25356.414 MB
Epoch: 2 (21.23s) 	Train: (l: 0.39, a: 0.82) 	Test: (l: 0.40, a: 0.82) 	Memory allocated: 25365.835 MB
Epoch: 3 (21.22s) 	Train: (l: 0.38, a: 0.83) 	Test: (l: 0.39, a: 0.83) 	Memory allocated: 25373.850 MB
Epoch: 4 (21.31s) 	Train: (l: 0.37, a: 0.84) 	Test: (l: 0.39, a: 0.83) 	Memory allocated: 25378.179 MB
Epoch: 5 (21.01s) 	Train: (l: 0.37, a: 0.84) 	Test: (l: 0.38, a: 0.83) 	Memory allocated: 25381.213 MB
